In [1]:
using QuantEcon

INFO: Precompiling module QuantEcon...


In [2]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [3]:
@everywhere using QuantEcon

In [4]:
nprocs()

5

In [5]:
workers()

4-element Array{Int64,1}:
 2
 3
 4
 5

In [6]:
@everywhere function lucas_tree_spec_rad(;beta=0.96,
                        gamma=2.0,
                        sigma=0.1,
                        b=0.0,
                        rho=0.9,
                        n=200)

    mc = tauchen(n, rho, sigma, b)  
    s = mc.state_values
    J = mc.p .* exp((1 - gamma) * s)
    return beta * maximum(abs(eigvals(J)))
end

In [7]:
lucas_tree_spec_rad(b=0.01)

1.2174480318956242

In [17]:
b_vals = collect(linspace(0.0, 0.5, 500));

In [18]:
out = Array(Float64, length(b_vals))
@time for i in eachindex(b_vals)
    out[i] = lucas_tree_spec_rad(b=b_vals[i])
end

 13.756786 seconds (46.00 k allocations: 674.538 MB, 0.19% gc time)


In [19]:
out_shared = SharedArray(Float64, length(b_vals))
@time @sync @parallel for i in eachindex(b_vals)
    out_shared[i] = lucas_tree_spec_rad(b=b_vals[i])
end

  3.740271 seconds (6.96 k allocations: 526.687 KB)


4-element Array{Any,1}:
 RemoteRef{Channel{Any}}(2,1,69)
 RemoteRef{Channel{Any}}(3,1,70)
 RemoteRef{Channel{Any}}(4,1,71)
 RemoteRef{Channel{Any}}(5,1,72)

In [11]:
out .== out_shared

20-element BitArray{1}:
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true